In [1]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsRegressor

df = sns.load_dataset("mpg")

k=5
horsepower=130
weight=3500
acceleration=13
pointt = [horsepower, weight, acceleration]


def Find_missing_rows(data, desired_columns):

    missing_values = data[data[desired_columns].isna().any(axis=1)]
    return missing_values

search_columns = ["horsepower", "weight","acceleration","mpg"]
print( Find_missing_rows(df, search_columns))

selected_c = ["horsepower", "weight", "acceleration","mpg"]

df_dropped = df.dropna(subset=selected_c)

X_train = df_dropped[["horsepower", "weight", "acceleration"]]
y= df_dropped['mpg']

def Euclidean_distance(point1, point2):

    return math.sqrt(sum((point1[i] - point2[i]) ** 2 for i in range(len(point1))))



k_values = list(range(1, 50))
cv_errors = []

for k in k_values:
    knn = KNeighborsRegressor(n_neighbors=k)
    errors = -cross_val_score(knn, X_train, y, cv=5, scoring='neg_mean_squared_error')
    cv_errors.append(errors.mean())


best_k = k_values[cv_errors.index(min(cv_errors))]


print(f'Best K value: {best_k}')

def SortedDistancewIndex(point, X_train):
    distances = []
    for index, row in X_train.iterrows():
        distance = Euclidean_distance(point, np.array(row))
        distances.append((distance,index))

    sorted_distances = sorted(distances, key=lambda x: x[0])

    return sorted_distances

def Weighted_knn_predict(point, X_train, y_train, k):


    sorted_distances_indices = SortedDistancewIndex(pointt,X_train)
    k_neighbors = sorted_distances_indices[:k]
    weighted_sum = 0
    total_weight = 0

    for distance, index in k_neighbors:
        weight = 1 / distance
        weighted_sum += weight * y_train.loc[index]
        total_weight += weight

    if total_weight == 0:
        return 0
    else:
        return weighted_sum / total_weight



predicted_value = Weighted_knn_predict(pointt, X_train, y, best_k)

print(" Predict with best K value:", predicted_value)
predicted_value = Weighted_knn_predict(pointt, X_train, y, k)
print("Predict :", predicted_value)


      mpg  cylinders  displacement  horsepower  weight  acceleration  \
32   25.0          4          98.0         NaN    2046          19.0   
126  21.0          6         200.0         NaN    2875          17.0   
330  40.9          4          85.0         NaN    1835          17.3   
336  23.6          4         140.0         NaN    2905          14.3   
354  34.5          4         100.0         NaN    2320          15.8   
374  23.0          4         151.0         NaN    3035          20.5   

     model_year  origin                  name  
32           71     usa            ford pinto  
126          74     usa         ford maverick  
330          80  europe  renault lecar deluxe  
336          80     usa    ford mustang cobra  
354          81  europe           renault 18i  
374          82     usa        amc concord dl  
Best K value: 35
 Predict with best K value: 18.28733723887729
Predict : 18.21703986750805
